In [1]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 50)

In [2]:
train  = pd.read_csv('../input/application_train.csv.zip')
train['is_train'] = 1

test  = pd.read_csv('../input/application_test.csv.zip')
test['is_train'] = 0

trte = pd.concat([train, test], ignore_index=True)

In [3]:
prev = pd.read_csv('../input/previous_application.csv.zip')
prev.sort_values('DAYS_DECISION', inplace=True)

In [4]:
user_id = pd.read_csv('../data/user_id_v4.csv.zip')

In [5]:
dup = user_id[user_id.duplicated('user_id', False)]
cnt = dup.groupby('user_id').size()
cnt.name = 'dup_cnt'
cnt = cnt.reset_index()
cnt['dup_id'] = range(cnt.shape[0])

dup = pd.merge(dup, cnt, on='user_id', how='left')
dup = pd.merge(dup, trte, on='SK_ID_CURR', how='left')
dup.sort_values(['user_id', 'DAYS_BIRTH'], ascending=[True, False], inplace=True)

dup['seq'] = 1
dup['seq'] = dup.groupby('user_id').seq.cumsum()-1

col = train.columns.tolist() + [c for c in dup.columns if c not in train.columns]
for c in ['dup_id', 'dup_cnt', 'user_id']:
    col.remove(c)
col = ['user_id'] + col

dup = dup[col]

In [36]:
dup[['user_id', 'SK_ID_CURR', 'TARGET', 'is_train', 'DAYS_BIRTH']]#.to_csv('dup_tbl.csv', index=False)

,user_id,SK_ID_CURR,TARGET,is_train,DAYS_BIRTH
0,2,100041,0.0,1,-12158
3012,2,181102,0.0,1,-12395
2605,3,170362,0.0,1,-17166
1,3,100043,0.0,1,-17199
2144,4,157771,0.0,1,-12435
2,4,100066,NaN,0,-12744
11604,5,417707,0.0,1,-7783
3,5,100072,0.0,1,-7907
12375,6,437195,0.0,1,-15292
4,6,100090,NaN,0,-15524


In [34]:
dup.groupby('user_id')['DAYS_BIRTH'].transform('diff')

0          NaN
3012       NaN
2605    -237.0
1       -237.0
2144       NaN
2          NaN
11604    -33.0
3        -33.0
12375      NaN
4          NaN
5       -309.0
2847    -309.0
6          NaN
9189       NaN
10971   -124.0
7       -124.0
8          NaN
9407       NaN
1221    -232.0
9       -232.0
10         NaN
2674       NaN
9875    -172.0
11      -172.0
4644       NaN
         ...  
12550    -79.0
12556      NaN
12661      NaN
12618      0.0
12575      0.0
12586      NaN
13021      NaN
12611      0.0
12767      0.0
12679      NaN
12839      NaN
12862      0.0
12741      0.0
12747      NaN
12942      NaN
12813   -120.0
12869   -120.0
12838      NaN
13076      NaN
12898   -224.0
12893   -224.0
12911      NaN
12906      NaN
12925     -1.0
13037     -1.0
Name: DAYS_BIRTH, Length: 13079, dtype: float64

In [7]:
sub_best = pd.read_csv('../data/stochastic_blending_v3-3_0.411rk7_0.304mw1_0.285t1_0.81031CV_0.3Adv.csv')

In [11]:
tmp = dup[dup.TARGET==0].user_id
tmp = dup[dup.user_id.isin(tmp)][dup.is_train==0].SK_ID_CURR

sub_best[sub_best.SK_ID_CURR.isin( tmp )]

/home/Kazuki/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,SK_ID_CURR,TARGET
36,100271,0.156819
452,103171,0.764340
1140,108045,0.986070
2470,117249,0.898757
2711,119014,0.614188
2828,119813,0.975751
2901,120388,0.172719
4304,130774,0.613737
4782,134494,0.973248
5165,137651,0.897936


In [14]:
user_id_test = dup[dup.is_train==0].user_id
dup_test = dup[dup.user_id.isin(user_id_test)]

In [15]:
user_id_test_t0 = dup_test[dup_test.TARGET==0].user_id
user_id_test_t1 = dup_test[dup_test.TARGET==1].user_id


In [16]:
ids_0 = dup[dup.user_id.isin(user_id_test_t0)][dup.is_train==0].SK_ID_CURR.tolist()
ids_1 = dup[dup.user_id.isin(user_id_test_t1)][dup.is_train==0].SK_ID_CURR.tolist()

/home/Kazuki/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/home/Kazuki/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [17]:
len(ids_0), len(ids_1)

(2443, 56)

In [18]:
len( set(ids_0) & set(ids_1) ), set(ids_0) & set(ids_1)

(1, {358955})

In [19]:
# remove 358955# remove 
ids_0.remove(358955)
ids_1.remove(358955)

# Swap

In [20]:
sub_best.sort_values('TARGET', inplace=True)

sub_0 = sub_best[sub_best.SK_ID_CURR.isin(ids_0)].reset_index(drop=True)
sub_1 = sub_best[sub_best.SK_ID_CURR.isin(ids_1)].reset_index(drop=True)
sub_other = sub_best[~sub_best.SK_ID_CURR.isin(ids_0+ids_1)].reset_index(drop=True)

In [21]:
sub_0

,SK_ID_CURR,TARGET
0,331032,0.000739
1,234695,0.001108
2,126822,0.001251
3,177424,0.001539
4,197224,0.001600
5,442541,0.002380
6,251267,0.002503
7,443039,0.002872
8,336791,0.003303
9,124362,0.003344


In [22]:
sub_1

,SK_ID_CURR,TARGET
0,100271,0.156819
1,120388,0.172719
2,233512,0.348351
3,268602,0.491609
4,288876,0.513232
5,401332,0.549750
6,310586,0.593673
7,145157,0.601756
8,130774,0.613737
9,119014,0.614188


In [23]:
target_all = sorted(sub_0.append(sub_1).TARGET.values)
target_0, target_1 = target_all[:sub_0.shape[0]], target_all[sub_0.shape[0]:]

In [24]:
len(target_0), len(target_1)

(2442, 55)

In [25]:
sub_0.TARGET = target_0
sub_1.TARGET = target_1

In [26]:
sub_0

,SK_ID_CURR,TARGET
0,331032,0.000739
1,234695,0.001108
2,126822,0.001251
3,177424,0.001539
4,197224,0.001600
5,442541,0.002380
6,251267,0.002503
7,443039,0.002872
8,336791,0.003303
9,124362,0.003344


In [27]:
sub_1

,SK_ID_CURR,TARGET
0,100271,0.975628
1,120388,0.975751
2,233512,0.976120
3,268602,0.976879
4,288876,0.977002
5,401332,0.978685
6,310586,0.979259
7,145157,0.979300
8,130774,0.979649
9,119014,0.979874


In [28]:
sub_new = pd.concat([sub_0, sub_1, sub_other])
sub_new.sort_values('TARGET', inplace=True)
sub_new.reset_index(drop=True, inplace=True)

sub_new.shape

(48744, 2)

In [29]:
sub_best.shape, sub_new.SK_ID_CURR.unique().shape

((48744, 2), (48744,))

In [30]:
sub_new.to_csv('../output/stochastic_blending_v3-2_0.308rk7_0.431mw1_0.261t1_0.81050CV_0.3Adv_Leak005.csv.gz', index=False, compression='gzip')